In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import datetime
import os
# import matplotlib.pyplot as plt
# import gensim
# import spacy

In [2]:
import spacy
# pip install -U spacy
# !python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.1 MB/s eta 0:00:0000:0100:01
--- Logging error ---
Traceback (most recent call last):
  File "/Users/paigelee/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Users/paigelee/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/Users/paigelee/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/Users/paigelee/opt/anaconda3/lib/python3.9/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Users/paigelee/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/Users/paigelee/opt/anaconda

In [23]:
df = pd.read_excel('Excel_files/Full_Poem_Dataset_9-22_0.xlsx')
df = df.drop('Unnamed: 0',axis=1)
records = df.to_dict('records')

print(len(df))
df.head()

2156


,Text,Author,Before or after,Source
0,Стихи о постсоветском человеке\n\n1.\nДесять з...,Александр Бараш,After,ROAR
1,2.\nПо капле выдавливать из себя \nсоветского ...,Александр Бараш,After,ROAR
2,"Дети\n\nСто шесть человек прибыли, тридцать че...",Александр Ланин,After,ROAR
3,Развод\n\nОднажды моя любимая дала пощёчину ре...,Александр Ланин,After,ROAR
4,Круциатус\n\nНевилл Лонгботтом стоит и смотрит...,Александр Ланин,After,ROAR


## Run Spacy

In [33]:
%%time
badIdxs = []
for i, rec in enumerate(records):
    if i % 100 == 0:
        print(f'{i}/{len(records)} parsed by Spacy.')
    try:
        lines = rec['Text'].split('\n')
        docLines = []
        for line in lines:
            docLines.append(nlp(line))
        rec['docLines'] = docLines
        # rec['doc'] = nlp(rec['Text'])
    except:
        badIdxs.append(i)
for i in badIdxs:
    records.pop(i)

0/2154 parsed by Spacy.
100/2154 parsed by Spacy.
200/2154 parsed by Spacy.
300/2154 parsed by Spacy.
400/2154 parsed by Spacy.
500/2154 parsed by Spacy.
600/2154 parsed by Spacy.
700/2154 parsed by Spacy.
800/2154 parsed by Spacy.
900/2154 parsed by Spacy.
1000/2154 parsed by Spacy.
1100/2154 parsed by Spacy.
1200/2154 parsed by Spacy.
1300/2154 parsed by Spacy.
1400/2154 parsed by Spacy.
1500/2154 parsed by Spacy.
1600/2154 parsed by Spacy.
1700/2154 parsed by Spacy.
1800/2154 parsed by Spacy.
1900/2154 parsed by Spacy.
2000/2154 parsed by Spacy.
2100/2154 parsed by Spacy.
CPU times: user 4min 17s, sys: 2.84 s, total: 4min 20s
Wall time: 4min 22s


In [26]:
# how many tokens?
bTotalTokens = 0
aTotalTokens = 0
for rec in records:
    if 'doc' not in rec:
        rec['doc'] = nlp(' ')
    if rec['Before or after'] == 'Before':
        bTotalTokens += len(rec['doc'])
    else:
        aTotalTokens += len(rec['doc'])
bTotalTokens, aTotalTokens

(191971, 230421)

## Text cleaning

In [27]:
def initializeRec(recText):
    if isinstance(recText, str):
        return recText.strip()
    else:
        return False

def skipLine(line, idx):
    if len(line.strip()) == 0:
        return False
    
    # throw hashtag line
    if line.strip()[0] == '#':
        return True
    
    # throw attribution line
    for attr in ['из личного','личный блог','источник:','авторский блог']:
        if attr in line.lower():
            return True
        
    # dots at the beginning
    matches = re.search("[\*\+\^-_][*+^-_= ]+", line.strip()) 
    if matches:
        return True
    
    # is none of it alphanumeric
    containsAlpha = False
    for char in line:
        if char.isalpha():
            containsAlpha = True
            break
    if not containsAlpha:
        return True
    
    return False


def processRec(rec):
    recText = initializeRec(rec['Text'])
    cleanLines = []
    if recText:
        # decide which lines to keep
        lines = recText.split('\n')
        for i, line in enumerate(lines):
            if skipLine(line, i):
                continue
            cleanLines.append(line)
    return '\n'.join(cleanLines)

## Testing out the NLP capabilities

In [32]:
rec = random.choice(records)
txt = rec['Text']
doc = rec['doc']

NERcounter = dict()
for ent in doc.ents:
    lemma = ent.lemma_
    entType = ent.label_
    NERcounter.setdefault(entType, dict())
    NERcounter[entType].setdefault(lemma, 0)
    NERcounter[entType][lemma] += 1
    
print(NERcounter)
print() 
print(txt)

{'PER': {'мол': 1, 'вова': 1, 'потрошитель вова\nсамый': 1, 'нюрнберг': 1}, 'ORG': {'оон': 2, 'омон': 1}}

После дождичка в четверг
Русский представитель
Обвинения отверг:
Вова-потрошитель
 
Мол не мог быть мясником
Вова мол не изверг –
По столу стучал в ООН
И слюною брызгал
 
Мол ни в чем не виноват
Потрошитель Вова
Самый лучший друг сирот
Муж примерный вдовам
 
Если в помыслах он чист
Это все детали
Ведь не может же чекист
Убивать в подвале
 
Из застенков до ООН
Не доходят крики
Если это все ОМОН
Мы к тому привыкли
 
Вам приносит русский мир
Вова-портошитель
Этот мир — большой сортир
Там вот и мочите
 
В перспективе Нюрнберг
И петля затянется
После дождичка в четверг
И в сухую пятницу


In [6]:
%%time

# NERcounter = dict()
# for rec in records:
#     lines = rec['Text'].split('\n')
#     for txt in lines:
#         doc = nlp(txt)
#         for ent in doc.ents:
#             lemma = ent.lemma_
#             entType = ent.label_
#             NERcounter.setdefault(entType, dict())
#             NERcounter[entType].setdefault(lemma, 0)
#             NERcounter[entType][lemma] += 1
            
# lst = []
# for key in NERcounter['LOC']:
#     lst.append((NERcounter['LOC'][key], key))
# sortedlst = sorted(lst, reverse=True)
# sortedlst[:20]

CPU times: user 36 µs, sys: 28 µs, total: 64 µs
Wall time: 143 µs
